# Model Comparison


## Imports and Config

In [3]:
from data.dataset import LeakAnomalyDetectionDataset
from data.data_loaders import get_autoencoder_data_loaders, get_classifier_data_loaders
from config import *
import torch
from sklearn.metrics import accuracy_score, det_curve
import matplotlib.pyplot as plt

leaks_dataset = LeakAnomalyDetectionDataset(normal_data_dir=NORMAL_DATA, anomalous_data_dir=ANOMALOUS_DATA)

## Load Best Models

## Model Comparisment
Multiple models are tested on the test set, to determine the final best model to use in production.

In [ ]:
_, _, test_loader = get_classifier_data_loaders(leaks_dataset, BATCH_SIZE)

In [ ]:
# False Alarm Rate

# Accuracy



# with torch.no_grad():
#   for x, (y_anomaly, y_distance) in test_loader:
#     anomaly, distance = model.forward(x)
#     fpr, fnr, thresholds = det_curve(y_anomaly, anomaly)
#     print(fpr, fnr, thresholds)